<a href="https://colab.research.google.com/github/nwang2208/Bioinformatics-Class-Projects/blob/mapping-reads/Project1bCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys, re
import numpy as np

In [ ]:
kmer = 10

todo
fix the actual finding of bases for deletion and substituion
take into account i have identified error type in misMatch



Opening Ref Genome

In [ ]:
ref=open('/content/drive/MyDrive/122data/project1b_1000000_reference_genome.fasta','r')
ref_lines=ref.readlines()
hre=re.compile('>(\S+)')
lre=re.compile('^(\S+)$')

ref_seq={}

for line in ref_lines:
        outh = hre.search(line)
        if outh:
                id=outh.group(1)
        else:
                outl=lre.search(line)
                if(id in ref_seq.keys()):
                        ref_seq[id] += outl.group(1)
                else:
                        ref_seq[id]  =outl.group(1)

ref_seq = ref_seq['genome']

In [ ]:
len(ref_seq)

1000

In [ ]:

start = 0
end = 0
transcript={}
for i in range(0, len(ref_seq)):
    end = i + kmer
    if(end > len(ref_seq)):
      break
    seq = ref_seq[i:end]
    if seq in transcript.keys():
      transcript[seq].append(i)
    else:
      transcript[seq] = [i]


In [ ]:
transcript

Opening reads


In [ ]:
r=open('/content/drive/MyDrive/122data/project1b_1000000_with_error_paired_reads.fasta','r')
reads_lines=r.readlines()

reads = {}
for line in reads_lines:
        outh = hre.search(line)
        if outh:
                id=outh.group(1)
        else:
                outl=lre.search(line)
                if(id in reads.keys()):
                        reads[id] += outl.group(1)
                else:
                        reads[id]  =outl.group(1)

In [ ]:
reads

Necessary Functions

In [ ]:
def scoring(positions): #helps alignRead

    #first find the ranges
    r1 = {}
    r2 = {}
    r3 = {}



    for x in range(0, len(positions)):
        for y in range(0, len(positions[x])):
            pos = positions[x][y]
            if len(str(pos)) == 3: #3 digit number
                rg = int(str(pos)[:2] + str(0))
            elif len(str(pos)) == 2: #2 digit number
                rg = int(str(pos)[0] + str(0))
            else: #single digit
                rg = pos

            if(x < 15):
                if rg in r1.keys():
                    r1[rg] += 1
                else:
                    r1[rg] = 1
            elif(x < 30):
                if rg in r2.keys():
                    r2[rg] += 1
                else:
                    r2[rg] = 1
            else:
                if rg in r3.keys():
                    r3[rg] += 1
                else:
                    r3[rg] = 1



    #now score
    final_positions = []
    scores = {}
    ranges = {}
    one_score = 0
    for x in range(0, len(positions)):
          if len(positions[x]) > 1: #there are multiple positions to choose from
              for i in range(0, len(positions[x])):
                  if(len(str(positions[x][i])) == 3): #3 digit number
                      kee = int(str(positions[x][i])[:2] + str(0))
                  elif len(str(positions[x][i])) == 2: #2 digit number
                      kee = int(str(positions[x][i])[0] + str(0))
                  else: #single digit number
                      kee = positions[x][i]

                  kee_before = kee - 10
                  kee_after = kee + 10

                  if(x < 15):
                      ranges = r1.copy()
                  elif(x < 30):
                      ranges = r2.copy()
                  else:
                      ranges = r3.copy()




                  if(kee_before in ranges.keys()):
                      score1 = ranges[kee_before]
                      one_score += score1
                  if(kee in ranges.keys()):
                      score2 = ranges[kee]
                      one_score += score2
                  if(kee_after in ranges.keys()):
                      score3 = ranges[kee_after]
                      one_score += score3

                  scores[positions[x][i]] = one_score
                  one_score = 0
              best_pos = maxDict(scores)
              final_positions.append(best_pos)
              scores = {}
          else:
              final_positions.append(positions[x][0])

    return(final_positions)

In [ ]:
def screening(final_positions, positions): #if there are consecutive positions and they accidentally chose the wrong ones, let's bring back incrememnting consecutive positions
    for r in range(1, len(final_positions)-1):
        num_interest = final_positions[r]
        before_num = final_positions[r - 1]
        after_num = final_positions[r + 1]
        if((num_interest != before_num+1) and (num_interest != after_num-1)): #both sides aren't good
            for i in range(0, len(positions[r])):
                if(positions[r][i] == before_num+1 and positions[r][i] == after_num-1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000
        elif(num_interest != before_num+1 and after_num == 10000): #after is a non match
            for i in range(0, len(positions[r])):
                if(positions[r][i] == before_num+1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000
        elif(num_interest != after_num-1 and before_num == 10000): #before is nonmatch
            for i in range(0, len(positions[r])):
                if(positions[r][i] == after_num-1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000
        elif(num_interest != before_num+1 and after_num != 10000 and before_num != 10000): #let's try to match our nums
            for i in range(0, len(positions[r])):
                if(positions[r][i] == before_num+1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000

    #now go through the list again to make sure no false 10000
    for a in range(1, len(final_positions) - 1):
        num_interest = final_positions[a]
        before_num = final_positions[a - 1]
        after_num = final_positions[a + 1]

        if (num_interest == 10000 and before_num != 10000): #before is a normal number
            for j in range(0, len(positions[a])):
                if(positions[a][j] == before_num + 1):
                    final_positions[a] = positions[a][j]
                    break
        elif(num_interest == 10000 and after_num != 10000): #after is a normal number
            for j in range(0, len(positions[a])):
                if(positions[a][j] == after_num - 1):
                    final_positions[a] = positions[a][j]
                    break




#we need to look at the last base too
    #if(final_positions[len(final_positions) - 1] != final_positions[len(final_positions)-2]):
        #for i in range(0, len(positions[r])):
                #if(positions[len(final_positions) - 1][i] == final_positions[len(final_positions)-2]): #ranges didn't catch things
                    #final_positions[len(final_positions) - 1] = positions[len(final_positions) - 1][i]
                    #break
                #else:
                    #final_positions[len(final_positions) - 1] = 10000


    return(final_positions)

In [ ]:
def maxDict(d): #helps(alignRead)
  return max(d, key = d.get)

In [ ]:
def alignRead(kmer, read): #final positions assigns a position to all bases, whether or not that is a good thing
    start = 0
    end = 0
    positions = []
    for i in range(0, len(read)): #go through the read
            end = i + kmer
            if(end > len(read)):
                break
            seq = read[i:end]
            if seq in transcript.keys(): #match with ref
                positions.append(transcript[seq])
            else:
                positions.append([10000])

    positions2 = scoring(positions)
    final_positions = screening(positions2, positions)


    return(final_positions)

In [ ]:
def splitList(final_positions):
    #screen out just wrong numbers
    split_list = {}
    count = 0
    for pos in final_positions:
        if(pos != 10000):
            if(count in split_list.keys()):
                split_list[count].append(pos)
            else:
                split_list[count] = [pos]
        elif(pos == 10000):
              count += 1
              if(count-1 in split_list.keys()):
                  a = 'nothing'
              else:
                  count -= 1

    return(split_list)






In [ ]:

def matchPos(split_list): #maybe just skip ahead of final_positions when you hit 1000 and see if you hit a good position a certain amount of kmers away
    match_pos = {}
    match_pos[0] = split_list[0].copy()
    match_key = 0

    #creating match_pos, a dict with only positions of bases that are right, this only works for substituion rn
    keeplast = 0
    prev_list = 0
    prev_index = 0
    for t in range(1, len(split_list)):
        firstofnext = split_list[t][0]
        if (keeplast != 1):
            prev_list = t-1
            prev_index = len(split_list[prev_list]) - 1
            keeplast = 0

        lastofprev = split_list[prev_list][prev_index]

        #both del and in look the same in matchPos
        if(0 <= firstofnext - lastofprev <= kmer*3):
            match_key += 1
            match_pos[match_key] = split_list[t]
            keeplast = 0
        else:
            keeplast = 1

    return(match_pos)


In [ ]:
def mismatchPos(match_pos, final_positions): #need to account for different types of error in the same read
        mis_pos = {}
        mis_key = 0
        count = 0
        errorCount = 0
        typeError = []

    #if(len(match_pos) >= 1):
        for u in range(1, len(match_pos)):
            firstofnext = match_pos[u][0] #actual position number
            prev_list = u-1
            prev_index = len(match_pos[prev_list]) - 1
            lastofprev = match_pos[prev_list][prev_index] #actual position number
            difference = firstofnext - lastofprev

            index_firstofnext = final_positions.index(firstofnext)
            index_lastofprev = final_positions.index(lastofprev)
            errorCount = index_firstofnext - index_lastofprev - 1
            if(errorCount == difference - 1): #substitution
                typeError.append('s')
                mis_index = final_positions.index(lastofprev)+1
                mis_pos[mis_index] = list(range(lastofprev+1, firstofnext))
            elif(errorCount == difference): #insertion
                typeError.append('i')
                mis_index = final_positions.index(lastofprev)+1
                mis_pos[mis_index] = list(range(lastofprev+1, firstofnext))
            elif(errorCount == difference - 2): #deletion
                typeError.append('d')
                mis_index = final_positions.index(lastofprev)+1
                mis_pos[mis_index] = list(range(lastofprev+1, firstofnext))





        return(mis_pos, typeError)

In [ ]:
def sub(key, value, read):
        base = ''
        to_add = ""
        final = []

        start = key
        pos_base = 0

        if(kmer < len(value) <= kmer + 1):
              #for the first sub
            for i in range(0, 4):
                read_seq = read[start:start+kmer-1] #cut off the last base, we will be adding different ones to find the real base
                if(i == 0):
                    base = 'T'
                elif(i == 1):
                    base = 'A'
                elif(i == 2):
                    base = 'G'
                elif(i == 3):
                    base = 'C'
                read_seq += base
                if(read_seq in transcript.keys()):
                    if(value[0] in transcript[read_seq]):
                        toadd = "S" + str(value[kmer-1]) + " " + base + " " + read[start+kmer-1]
                        final.append(toadd)
                        break


                toadd = "S" + str(value[len(value)-1]) + " " + ref_seq[value[len(value)-1]] + " " + read[start+len(value)-1]
                final.append(toadd)

        else:
            subpos = int(len(value)/kmer)
            for x in range(0, subpos):
                for i in range(0, 4):
                    read_seq = read[start:start+kmer-1] #cut off the last base, we will be adding different ones to find the real base
                    if(i == 0):
                        base = 'T'
                    elif(i == 1):
                        base = 'A'
                    elif(i == 2):
                        base = 'G'
                    elif(i == 3):
                        base = 'C'
                    read_seq += base
                    if(read_seq in transcript.keys()):
                        if(value[pos_base] in transcript[read_seq]):
                            toadd = "S" + str(value[pos_base + kmer - 1]) + " " + base + " " + read[start+kmer-1]
                            final.append(toadd)
                            break
                start += kmer
                pos_base += kmer

        final = tuple(final)
        return(final)

In [ ]:
def dele(key, value):
        final = []
        toadd = ""

        miss_pos = value[len(value)-1]
        base = ref_seq[miss_pos]
        toadd = "D" + str(miss_pos) + " " + base
        final.append(toadd)

        final = tuple(final)
        return(final)


In [ ]:
def inser(key, value, final_positions, read):
        toadd = ""
        final = []

        miss_pos = value[len(value)-1]
        mark_pos = miss_pos + 1
        miss_index = final_positions.index(mark_pos) - 1
        base = read[miss_index]
        toadd = "I" + str(miss_pos) + " " + base
        final.append(toadd)

        final = tuple(final)
        return(final)


In [ ]:
def mutation(typeError, mis_pos, final_positions, read):
    count = 0
    error = ''
    for key, value in mis_pos.items():
            error = typeError[count]
            if(error == 's'):
                su = sub(key, value, read)
                for el in su:
                    '''if(el == 'S129 C A'):
                        print(read)
                    if(el == 'S141 G A'):
                        print(read)'''
                    if(el in mutation_list.keys()):
                        mutation_list[el] += 1
                    else:
                        mutation_list[el] = 1
            elif(error == 'i'):
                ins = inser(key, value, final_positions, read)
                for el in ins:
                    if(el == 'I544 G'):
                        print(read)

                    if(el in mutation_list.keys()):
                        mutation_list[el] += 1
                    else:
                        mutation_list[el] = 1
            elif(error == 'd'):
                de = dele(key, value)
                for el in de:
                    if(el in mutation_list.keys()):
                        mutation_list[el] += 1
                    else:
                        mutation_list[el] = 1

            count += 1


Compile the mutation list

In [ ]:
mutation_list = {}

In [ ]:
positions_dict = {} #position number in ref_seq | reads that map to it correctly from match_pos

In [ ]:

for key, value in reads.items():
    if(len(value) >= kmer):
        final_positions = alignRead(kmer, value)

        #check 129
        '''if(119 in final_positions and key!= 'read_46/1' and key!= 'read_19/1' and key!= 'read_38/1'):
            print(key)
            break'''

        split_list = splitList(final_positions)
        if(split_list != {}): #the entire read is not an error
            match_pos = matchPos(split_list)

            mis_pos = mismatchPos(match_pos, final_positions)[0]
            typeError = mismatchPos(match_pos, final_positions)[1]

            if(typeError != ''):
                mutation(typeError, mis_pos, final_positions, value)

            #build positions_dict
            for key2, value2 in match_pos.items():
                for el in value2:
                    if(el in positions_dict.keys()):
                        positions_dict[el].append(key)
                    else:
                        positions_dict[el] = [key]







In [ ]:
#screens out errors in the reads
final_mutation_list = {}
final_mutation_list = mutation_list.copy()
for key, value in mutation_list.items():
    space_index = key.find(' ')
    pos_number = int(key[1:space_index])
    if(pos_number in positions_dict and len(positions_dict[pos_number]) > value): #hmmmmm
        del final_mutation_list[key]
    elif mutation_list[key] ==1:
        del final_mutation_list[key]



In [ ]:
final_mutation_list

In [ ]:
mutation_keys = list(final_mutation_list.keys())
mutation_keys.sort()
#mutation_keys

Making a csv file

In [ ]:
predictions = []
for el in mutation_keys:
    el = ">" + el
    predictions.append(el)

In [ ]:
predictions

In [ ]:
np.savetxt("predictions.csv",
           predictions,
           delimiter =", ",  # Set the delimiter as a comma followed by a space
           fmt ='% s')